In [ ]:
cd ..

In [ ]:
#plot&calc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.ticker as ticker

import scipy
import scipy.optimize as optimize
import sklearn.metrics as metrics
from sklearn.metrics import r2_score

#pickle
import pickle

#ファイル管理系
import os
import glob

def make_dirs(path):
    try:
        os.makedirs(path)
        print("Correctly make dirs")
    except:
        print("Already exist or fail to make dirs")
        
def std_mean(t):
    value_return = np.sqrt(np.average((t)**2))
    return value_return
def score_mean(t):
    value_return = np.average(t)
    return value_return

plt.rcParams["font.size"] = 25
plt.rcParams['font.sans-serif'] = ['Arial']

In [ ]:
def make_dirs(path):
    try:
        os.makedirs(path)
        print("Correctly make dirs")
    except:
        print("Already exist or fail to make dirs")

def Spidergram_simple(data, scale, legend, color, style, label):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length
    
    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)

    if(scale == "log"):
        plt.yscale("log")
    
    for num in range(sample_nun):
        if num ==0:
            plt.errorbar(t, data.values[num], color = color, linestyle=style, label = label, linewidth = 3.0)
        else:
            plt.errorbar(t, data.values[num], color = color, linestyle=style, linewidth = 3.0)
     
    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_marker(data, immobile_elem, color, markeredgecolor, style, size):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length
    
    ######## define marker elem
    marker_row = []
    for immobile_elem_now in immobile_elem:
        num = list(columns.values).index(immobile_elem_now)
        marker_row.append(num)
    marker_row.sort() # compare with t

    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define cycle of plotting     

    ######## marker plot
    for num in range(sample_nun):
        plt.errorbar(t, data.values[num], color = color, linestyle="None", marker=style, \
                     markersize=size, markevery=marker_row, markeredgecolor=markeredgecolor, markeredgewidth=2.5)
    ######## marker plot

    

def Spidergram_error(data, model_score,scale, legend, color, style, label):
    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    sample_nun = len(data.index)
    data=data.astype(float)
    ######## Define X(columns) and X length

    ################################# To plot Error bar
    #score_elem
    model_score = model_score[columns]
    
    #error_bar_calculation log_scale
    error_plus_log = data.apply(lambda x : np.log10(x)) + model_score
    error_minus_log = data.apply(lambda x : np.log10(x)) - model_score
    raw_log = data.apply(lambda x : np.log10(x)).values
    #error_bar_calculation normal_scale
    error_plus = error_plus_log.apply(lambda x : 10**x) - 10**raw_log
    error_minus = 10**raw_log - error_minus_log.apply(lambda x : 10**x)
    error = pd.concat([error_minus, error_plus], axis = 0).values
    ################################# To plot Error bar

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")
    
    for num in range(sample_nun):
        if num ==0:
            plt.errorbar(t, data.values[num], yerr = error, color = color, linestyle=style, label = label, capsize=5, linewidth = 3.0)
        else:
            plt.errorbar(t, data.values[num], yerr = error, color = color, linestyle=style, capsize=5, linewidth = 3.0)
     
    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_fill_between(data, scale, legend, color, style, label, alpha):

    ######## Define X(columns) and X length
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length

    ######## Define MinMax
    # Min Maxを求めて， 上限下限の値を決める
    data_max = pd.DataFrame(data.max()).T
    data_min = pd.DataFrame(data.min()).T
    ######## Define MinMax

    ######## Define cycle of plotting 
    sample_nun = len(data.index)
    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")

    plt.fill_between(t, data_max.values[0], data_min.values[0], color = color, linestyle=style, label = label, alpha = alpha)

    if (legend == "on"):
        plt.legend(data.index)

def Spidergram_fill_immobile(columns, immobile_elem, color, alpha):
    ######## Define X(columns) and X length
    length = len(columns)
    t = np.arange(0,length, 1)
    ######## Define X(columns) and X length

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    ######## Define X axis and label

    ######## Define fill index and fill X axis
    # 箱を準備
    elem_fill_x = []
    for elem in immobile_elem:
        # 一致しているインデックス番号を代入
        #elem_fill_x.append(list(columns).index(elem))
        num = list(columns).index(elem)
        plt.axvspan(num-0.45, num+0.45, color = color, alpha = alpha)
    ######## Define fill index

def Spidergram_ragne_as_error_bar(data, range, scale, legend, color, style, label):
    columns = data.columns
    length = data.values.shape[1]
    t = np.arange(0,length, 1)
    sample_nun = len(data.index)
    data=data.astype(float)
    ######## Define X(columns) and X length

    ################################# To plot Error bar
    #score_elem
    error_plus_log = (data*range[0]).apply(lambda x : np.log10(x))
    error_minus_log = (data*range[1]).apply(lambda x : np.log10(x))
    raw_log = data.apply(lambda x : np.log10(x)).values

    #error_bar_calculation normal_scale
    error_plus = error_plus_log.apply(lambda x : 10**x) - 10**raw_log
    error_minus = 10**raw_log - error_minus_log.apply(lambda x : 10**x)
    error = pd.concat([error_minus, error_plus], axis = 0)#values
    ################################# To plot Error bar

    ######## Define X axis and label
    plt.xticks(t, columns)
    plt.xticks(rotation = 70)
    if(scale == "log"):
        plt.yscale("log")
    plt.errorbar(t, data.values.T, yerr = error.values, color = color, linestyle=style, capsize=5, linewidth = 3.0)

In [ ]:
path = 'Kelley_Data/NGBoost/NGBoost_801SUPER/'
Raw_data = pd.read_csv(path+"Altered/PM_applied_with.csv", index_col=0)
mobility_data = pd.read_csv(path+"Altered/Element_mobility_by_PRM_No_database_info.csv", index_col=0)
uncertanty_data = pd.read_csv(path+"Altered/Predictive_uncertanty_No_database_info.csv", index_col=0)
predict_data = pd.read_csv(path+"Altered/Protolith_comp_by_PRM_(PM_norm)_No_database_info.csv", index_col=0)

Host_data = pd.read_csv(path+"HOST/PM_applied_with.csv", index_col=0)
Host_data = Host_data.loc[Host_data.index[:-2]]

Kelly2003_Altered = Raw_data.index

In [ ]:
immobile_elem = ["Zr", "Th", "Ti", "Nb"]
mobile_elem_list = ['Rb', 'Ba', 'U', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Y', 'Yb', 'Lu',] 
list_elem = ['Rb', 'Ba', 'Th', 'U', 'Nb', 'K', 'La', 'Ce', 'Pb', 'Sr', 'Nd', 'Zr', 'Ti', 'Y', 'Yb', 'Lu']

uncertanty_data[immobile_elem] = 0

In [ ]:
############################################################################ Make dir and path
# make dirs
path_spidergram_examples = path + "Spidergram_predict_compare_to_Protolish"
make_dirs(path_spidergram_examples)
############################################################################ Make dir and path

# 論文ごとにホストロックを設定し，図を出力していく。
for index_name in Kelly2003_Altered:
    print(index_name)
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    ########################################## Plot composition data
    # Fill immobile_element
    Spidergram_fill_immobile(Host_data[list_elem].columns, immobile_elem, '#ecc06f', 0.18)

    # Plot Host Rock (MinMax Width Plot)
    Spidergram_fill_between(Host_data[list_elem], "log", "off","#6fecc0", "-", "off", 0.4)


    # Plot Raw Data(Altered or Metamorphic)
    Spidergram_simple(pd.DataFrame(Raw_data[list_elem].loc[index_name]).T, "log", "off","#344c5c", "--", "off")
    Spidergram_marker(pd.DataFrame(Raw_data[list_elem].loc[index_name]).T, immobile_elem, '#f08575', '#344c5c', 'd', 16)
    # Plot Predicted Data and Error bar
    Spidergram_error(pd.DataFrame(predict_data[list_elem].loc[index_name]).T, uncertanty_data.loc[index_name], "log", "off","#f08575", "-", "off")
    ########################################## Plot composition data

    ##### Define setting and savefig
    plt.ylabel('Sample / PM')
    plt.ylim(0.1, 100)
    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)

    plt.savefig(path_spidergram_examples + "/"+ index_name + '.pdf', bbox_inches='tight')
    plt.show()

###################################### 論文ごとにホストロックを設定し，図を出力していく。

In [ ]:
# 論文ごとにホストロックを設定し，図を出力していく。
for index_name in Kelly2003_Altered:
    print(index_name)
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    Mobile_data_now = pd.DataFrame(Raw_data[list_elem].loc[index_name] / predict_data[list_elem].loc[index_name]).T
    # Fill immobile_element
    Spidergram_fill_immobile(Host_data[list_elem].columns, immobile_elem, '#ecc06f', 0.18)
    #Spidergram_simple(pd.DataFrame(Raw_data[list_elem].loc[index_name] / Host_data[list_elem].loc['801C-46R-1-45-51']).T, "log", "off","#344c5c", "--", "off")
    Spidergram_marker(Mobile_data_now[list_elem], immobile_elem, '#f08575', '#344c5c', 'd', 16)
    Spidergram_error(Mobile_data_now[list_elem], uncertanty_data.loc[index_name], "log", "off","#f08575", "-", "off")
    plt.hlines(y=1, xmin=0, xmax=len(Mobile_data_now.columns)-1, color = "#344c5c", linestyle='--',)
    ##### Define setting and savefig
    plt.yscale('log')
    plt.ylim(0.1, 50)
    plt.ylabel('Sample/Protolith')
    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)
    
    plt.savefig(path_spidergram_examples + "/"+ index_name + '_Mobile.pdf', bbox_inches='tight')
    plt.show()
    ##########################################

In [ ]:
# 論文ごとにホストロックを設定し，図を出力していく。
for index_name in ["801_SUPER"]:
    print(index_name)
    # Fontsize
    plt.rcParams["font.size"] = 21
    plt.rcParams['font.family'] = 'Arial'
    Mobile_data_now = pd.DataFrame(Raw_data[list_elem].loc[index_name] / predict_data[list_elem].loc[index_name]).T
    # Fill immobile_element
    Spidergram_fill_immobile(Host_data[list_elem].columns, immobile_elem, '#ecc06f', 0.18)
    Spidergram_simple(pd.DataFrame(Raw_data[list_elem].loc[index_name] / Host_data[list_elem].loc['801C-46R-1-45-51']).T, "log", "off","#344c5c", "--", "off")
    Spidergram_marker(Mobile_data_now[list_elem], immobile_elem, '#f08575', '#344c5c', 'd', 16)
    Spidergram_error(Mobile_data_now[list_elem], uncertanty_data.loc[index_name], "log", "off","#f08575", "-", "off")
    plt.hlines(y=1, xmin=0, xmax=len(Mobile_data_now.columns)-1, color = "#344c5c", linestyle='--',)
    ##### Define setting and savefig
    plt.yscale('log')
    plt.ylim(0.1, 50)
    plt.ylabel('Sample/Protolith')
    plt.tick_params(which='both', direction='in',bottom=True, left=True, top=True, right=True)
    plt.tick_params(which = 'major', length = 7.5, width = 2)
    plt.tick_params(which = 'minor', length = 4, width = 1)
    
    plt.savefig(path_spidergram_examples + "/"+ index_name + '_Mobile_.pdf', bbox_inches='tight')
    plt.show()
    ##########################################